In [ ]:
#Generate sequences for all mutations in deep mutational scan covid spike RBD experiment 
#! pip install Bio
import pandas as pd
import Bio
from Bio import SeqIO
from Bio import SeqRecord
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO.Interfaces import MutableSeq
import tabulate

url = 'https://media.githubusercontent.com/media/jbloomlab/SARS-CoV-2-RBD_DMS_variants/main/results/final_variant_scores/final_variant_scores.csv'
df = pd.read_csv(url) 

strains = ['Beta', 'E484K', 'N501Y', 'Wuhan-Hu-1', 'Delta']
wtSeq = [] 
for strain in strains: 
  seq = ''
  for i in range(0, 4019, 20): 
    z = df[df.target == strain].reset_index() 
    seq+=(z.loc[i]['wildtype']) 
  seqRec = SeqRecord(
      Seq(seq),
      id = strain + "WT"
  )
  wtSeq.append(seqRec)
#print(wtSeq)
 
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] 
mutRec =[]
for rec in wtSeq:
  for position in range(0,201): 
    for x in amino_acids: 
      sequence = MutableSeq(rec.seq)  
      sequence[position]= x 
      seqRec = SeqRecord(
          Seq(sequence),
          id = rec.id + '_' + str(position + 331) + "_" + x
      )
      mutRec.append(seqRec)
with open("out.fas","w") as out_fh:
  SeqIO.write(mutRec, out_fh, 'fasta')

#linking sequences to binding data
seq_records = list(SeqIO.parse("/content/out.fas", "fasta"))
seq_bind_data = []
for i in range(0, 20100):
  seq_dict = {'id' : seq_records[i].id, 'sequence' : seq_records[i].seq, 'bind' : df.loc[i]['bind'], 'delta_bind' : df.loc[i]['delta_bind']}
  seq_bind_data.append(seq_dict)

#for item in seq_bind_data:
    #print(item['id'],"\t",item['sequence'],"\t",item['bind'],"\t",item['delta_bind'])
